In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import visdom
import matplotlib.pyplot as plt

import mxnet as mx
from mxnet import gluon
from mxnet import autograd
from mxnet import image

import sys
sys.path.append('../../resuneta/src')
sys.path.append('../../resuneta/nn/loss')
sys.path.append('../../resuneta/models')
sys.path.append('../../')
sys.path.append('../MXNet-ResUNeta/')

from bound_dist import get_distance, get_boundary
from loss import Tanimoto_wth_dual
from resunet_d6_causal_mtskcolor_ddist import *
from resunet_d7_causal_mtskcolor_ddist import *

from datasets import *

In [2]:
def dice_coef(x, y):
    if type(x).__module__ == 'numpy':
        intersection = np.logical_and(x, y)
        return 2. * np.sum(intersection) / (np.sum(x) + np.sum(y))
    else:
        intersection = mx.ndarray.op.broadcast_logical_and(x, y)
        return 2. * mx.nd.sum(intersection) / (mx.nd.sum(x) + mx.nd.sum(y))

In [3]:
def visdom_visualize_batch(vis, img, extent, boundary, distance,
                           extent_pred, boundary_pred, distance_pred,
                           hsv, hsv_pred, title="Train images", n_channels=3):

    img, extent, boundary, distance = img.asnumpy(), extent.asnumpy(), boundary.asnumpy(), distance.asnumpy()
    extent_pred, boundary_pred = extent_pred.asnumpy(), boundary_pred.asnumpy()
    distance_pred, hsv, hsv_pred = distance_pred.asnumpy(), hsv.asnumpy(), hsv_pred.asnumpy()

    # put everything in one window
    batch_size, nchannels, nrows, ncols = img.shape
    padding = 10
    
    items = []
    for i in range(img.shape[1] // n_channels):
        items.append(img[:,i*n_channels:(i+1)*n_channels])
    for i in range(hsv_pred.shape[1] // n_channels):
        items.append(hsv_pred[:,i*n_channels:(i+1)*n_channels])
    items = items + [extent, extent_pred, boundary, boundary_pred, distance, distance_pred]
    result = np.zeros((3, len(items)*nrows + (len(items)-1)*padding, batch_size*ncols + (batch_size-1)*padding))

    for j, item in enumerate(items):

        if item.shape[1] == 1:
            item = np.tile(item, (1,3,1,1)) * 255.

        if (j >= img.shape[1] // n_channels) and \
        (j < img.shape[1] // n_channels + hsv_pred.shape[1] // n_channels): # convert HSV to RGB
            item = np.moveaxis(item, 1, -1) * 255.
            for i in range(batch_size):
                item[i] = cv2.cvtColor(item[i].astype(np.uint8), cv2.COLOR_HSV2RGB)
            item = np.moveaxis(item, -1, 1)
            
        for i in range(batch_size):
            result[:, j*(nrows+padding):(j+1)*nrows+j*padding, i*(ncols+padding):(i+1)*ncols+i*padding] = item[i]
    vis.images(result, nrow=1, win=title, opts={'title': title})


In [4]:
def train_model(train_dataloader, model, tanimoto_dual, trainer, epoch, args):
    
    # initialize metrics
    cumulative_loss = 0
    accuracy = mx.metric.Accuracy()
    f1 = mx.metric.F1()
    mcc = mx.metric.MCC()
    dice = mx.metric.CustomMetric(feval=dice_coef, name="Dice")
    
    # training set
    for batch_i, (img, extent, boundary, distance, hsv) in enumerate(
        tqdm(train_dataloader, desc='Training epoch {}'.format(epoch))):
        
        with autograd.record():

            img = img.as_in_context(mx.gpu(args['gpu']))
            extent = extent.as_in_context(mx.gpu(args['gpu']))
            boundary = boundary.as_in_context(mx.gpu(args['gpu']))
            distance = distance.as_in_context(mx.gpu(args['gpu']))
            hsv = hsv.as_in_context(mx.gpu(args['gpu']))
            
            logits, bound, dist, convc = model(img)
            
            # multi-task loss
            # TODO: wrap this in a custom loss function / class
            loss_extent = mx.nd.sum(1 - tanimoto_dual(logits, extent))
            loss_boundary = mx.nd.sum(1 - tanimoto_dual(bound, boundary))
            loss_distance = mx.nd.sum(1 - tanimoto_dual(dist, distance))
            loss_hsv = mx.nd.sum(1 - tanimoto_dual(convc, hsv))
            
            loss = 0.25 * (loss_extent + loss_boundary + loss_distance + loss_hsv)
            
        loss.backward()
        trainer.step(args['batch_size'])
        
        # update metrics based on every batch
        cumulative_loss += mx.nd.sum(loss).asscalar()
        # accuracy
        extent_predicted_classes = mx.nd.ceil(logits[:,[0],:,:] - 0.5)
        accuracy.update(extent, extent_predicted_classes)
        # f1 score
        prediction = logits[:,0,:,:].reshape(-1)
        probabilities = mx.nd.stack(1 - prediction, prediction, axis=1)
        f1.update(extent.reshape(-1), probabilities)
        # MCC metric
        mcc.update(extent.reshape(-1), probabilities)
        # Dice score
        dice.update(extent, extent_predicted_classes)
        # TODO: eccentricity
        # TODO: ...
        
        if batch_i % args['visdom_every'] == 0:
            visdom_visualize_batch(args['visdom'], img, extent, boundary, distance,
                                   logits, bound, dist, hsv, convc)

    return cumulative_loss, accuracy, f1, mcc, dice


In [5]:
def evaluate_model(val_dataloader, model, tanimoto_dual, epoch, args):
    
    # initialize metrics
    cumulative_loss = 0
    accuracy = mx.metric.Accuracy()
    f1 = mx.metric.F1()
    mcc = mx.metric.MCC()
    dice = mx.metric.CustomMetric(feval=dice_coef, name="Dice")
    
    # validation set
    for batch_i, (img, extent, boundary, distance, hsv) in enumerate(
        tqdm(val_dataloader, desc='Validation epoch {}'.format(epoch))):

        img = img.as_in_context(mx.gpu(args['gpu']))
        extent = extent.as_in_context(mx.gpu(args['gpu']))
        boundary = boundary.as_in_context(mx.gpu(args['gpu']))
        distance = distance.as_in_context(mx.gpu(args['gpu']))
        hsv = hsv.as_in_context(mx.gpu(args['gpu']))

        logits, bound, dist, convc = model(img)
            
        # multi-task loss
        # TODO: wrap this in a custom loss function / class
        loss_extent = mx.nd.sum(1 - tanimoto_dual(logits, extent))
        loss_boundary = mx.nd.sum(1 - tanimoto_dual(bound, boundary))
        loss_distance = mx.nd.sum(1 - tanimoto_dual(dist, distance))
        loss_hsv = mx.nd.sum(1 - tanimoto_dual(convc, hsv))

        loss = 0.25 * (loss_extent + loss_boundary + loss_distance + loss_hsv)
        
        # update metrics based on every batch
        cumulative_loss += mx.nd.sum(loss).asscalar()
        # accuracy
        extent_predicted_classes = mx.nd.ceil(logits[:,[0],:,:] - 0.5)
        accuracy.update(extent, extent_predicted_classes)
        # f1 score
        prediction = logits[:,0,:,:].reshape(-1)
        probabilities = mx.nd.stack(1 - prediction, prediction, axis=1)
        f1.update(extent.reshape(-1), probabilities)
        # MCC metric
        mcc.update(extent.reshape(-1), probabilities)
        # Dice score
        dice.update(extent, extent_predicted_classes)
        # TODO: eccentricity
        # TODO: ...
        
        if batch_i % args['visdom_every'] == 0:
            visdom_visualize_batch(args['visdom'], img, extent, boundary, distance,
                                   logits, bound, dist, hsv, convc, title="Val images")
        
    return cumulative_loss, accuracy, f1, mcc, dice

# New 10k samples in France

In [6]:
def run_france(splits_path='../data/splits/sherrie10k_planetImagery_splits.csv',
               epochs=100, lr=0.001, lr_decay=None, n_filters=16, batch_size=8,
               n_classes=1, model_type='resunet-d6', month='april', 
               codes_to_keep=list(np.arange(1,29, dtype=np.uint8)), 
               image_suffix='.tif', gpu_id=0):
    
    
    # Set up names of directories and paths for saving
    if len(codes_to_keep) >= 24:
        classes = 'all'
    elif codes_to_keep == list(range(1,10)) + [11,14,15,16,17,18,19,24,25,26,27]:
        classes = 'notreecrops'
    else:
        classes = 'subset'
    folder_name = model_type+'_'+month+'_class-'+classes+'_nfilter-'+str(n_filters)+'_bs-'+str(batch_size)+'_lr-'+str(lr)
    if lr_decay:
        folder_name = folder_name + '_lrdecay-'+str(lr_decay)
    save_path = os.path.join('../experiments/france/sherrie10k/', folder_name)
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    save_model_name = os.path.join(save_path, "model.params")
    
    # Directories for loading data
    month_dict = {'april': '2019_04', 'october': '2019_10', 'january': '2019_01'}
    month_folder = month_dict[month]
    image_directory = os.path.join(
        '../data/planet/france/sherrie10k/monthly_mosaics_renamed_clipped_merged/', month_folder)
    label_directory = '../data/planet/france/sherrie10k/extent_labels/'

    # Visdom
    env_name = 'france_sherrie10k_'+folder_name
    vis = visdom.Visdom(port=8097, env=env_name)
    
    # Arguments
    args = {}
    args['batch_size'] = batch_size
    args['gpu'] = gpu_id
    args['visdom'] = vis
    args['visdom_every'] = 20

    # Load train/val/test splits
    splits_df = pd.read_csv(splits_path)
    splits_df['image_id'] = splits_df['image_id'].astype(str).str.zfill(5)
    train_names = splits_df[splits_df['fold'] == 'train']['image_id'].values
    val_names = splits_df[splits_df['fold'] == 'val']['image_id'].values
    test_names = splits_df[splits_df['fold'] == 'test']['image_id'].values

    train_dataset = PlanetDatasetWithClasses(
        image_directory, label_directory, fold='train', image_names=train_names, classes=codes_to_keep,
        image_suffix=image_suffix)
    val_dataset = PlanetDatasetWithClasses(
        image_directory, label_directory, fold='val', image_names=val_names, classes=codes_to_keep,
        image_suffix=image_suffix)
    test_dataset = PlanetDatasetWithClasses(
        image_directory, label_directory, fold='test', image_names=test_names, classes=codes_to_keep,
        image_suffix=image_suffix)

    train_dataloader = gluon.data.DataLoader(train_dataset, batch_size=batch_size)
    val_dataloader = gluon.data.DataLoader(val_dataset, batch_size=batch_size)
    test_dataloader = gluon.data.DataLoader(test_dataset, batch_size=batch_size)

    # define model
    if model_type == 'resunet-d6':
        model = ResUNet_d6(_nfilters_init=n_filters, _NClasses=n_classes)
    elif model_type == 'resunet-d7':
        model = ResUNet_d7(_nfilters_init=n_filters, _NClasses=n_classes)
    model.initialize()
    model.hybridize()
    model.collect_params().reset_ctx(mx.gpu(args['gpu']))

    # define loss function
    tanimoto_dual = Tanimoto_wth_dual()
    if lr_decay:
        schedule = mx.lr_scheduler.FactorScheduler(step=1, factor=lr_decay)
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr, lr_scheduler=schedule)
    else:
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr)
    trainer = gluon.Trainer(model.collect_params(), optimizer=adam_optimizer)

    # containers for metrics to log
    train_metrics = {'train_loss': [], 'train_acc': [], 'train_f1': [], 
                     'train_mcc': [], 'train_dice': []}
    val_metrics = {'val_loss': [], 'val_acc': [], 'val_f1': [], 
                   'val_mcc': [], 'val_dice': []}
    best_mcc = 0.0

    # training loop
    for epoch in range(1, epochs+1):

        # training set
        train_loss, train_accuracy, train_f1, train_mcc, train_dice = train_model(
            train_dataloader, model, tanimoto_dual, trainer, epoch, args)

        # training set metrics
        train_loss_avg = train_loss / len(train_dataset)
        train_metrics['train_loss'].append(train_loss_avg)
        train_metrics['train_acc'].append(train_accuracy.get()[1])
        train_metrics['train_f1'].append(train_f1.get()[1])
        train_metrics['train_mcc'].append(train_mcc.get()[1])
        train_metrics['train_dice'].append(train_dice.get()[1])

        # validation set
        val_loss, val_accuracy, val_f1, val_mcc, val_dice = evaluate_model(
            val_dataloader, model, tanimoto_dual, epoch, args)

        # validation set metrics
        val_loss_avg = val_loss / len(val_dataset)
        val_metrics['val_loss'].append(val_loss_avg)
        val_metrics['val_acc'].append(val_accuracy.get()[1])
        val_metrics['val_f1'].append(val_f1.get()[1])
        val_metrics['val_mcc'].append(val_mcc.get()[1])
        val_metrics['val_dice'].append(val_dice.get()[1])

        print("Epoch {}:".format(epoch))
        print("    Train loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            train_loss_avg, train_accuracy.get()[1], train_f1.get()[1], train_mcc.get()[1], train_dice.get()[1]))
        print("    Val loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            val_loss_avg, val_accuracy.get()[1], val_f1.get()[1], val_mcc.get()[1], val_dice.get()[1]))

        # save model based on best MCC metric
        if val_mcc.get()[1] > best_mcc:
            model.save_parameters(save_model_name)
            best_mcc = val_mcc.get()[1]

        # save metrics
        metrics = pd.concat([pd.DataFrame(train_metrics), pd.DataFrame(val_metrics)], axis=1)
        metrics.to_csv(os.path.join(save_path, 'metrics.csv'), index=False)

        # visdom
        vis.line(Y=np.stack([train_metrics['train_loss'], val_metrics['val_loss']], axis=1), 
                 X=np.arange(1, epoch+1), win="Loss", 
                 opts=dict(legend=['train loss', 'val loss'], markers=False, title="Losses",
                           xlabel="Epoch", ylabel="Loss")
                )
        vis.line(Y=np.stack([train_metrics['train_mcc'], val_metrics['val_mcc']], axis=1), 
                 X=np.arange(1, epoch+1), win="MCC", 
                 opts=dict(legend=['train MCC', 'val MCC'], markers=False, title="MCC",
                           xlabel="Epoch", ylabel="MCC")
                )


In [7]:
# ============================ #
# user-specified hyperparameters
# ============================ #
epochs = 100
lr = 0.001
lr_decay = None
n_filters = 16
batch_size = 8
n_classes = 1
model_type = 'resunet-d6'
month = 'october'
codes_to_keep = list(range(1,10)) + [11,14,15,16,17,18,19,24,25,26,27]
gpu_id = 0
# ============================ #

run_france(splits_path='../data/splits/sherrie10k_planetImagery_splits_20x20.csv',
           epochs=epochs, lr=lr, lr_decay=lr_decay, n_filters=n_filters, batch_size=batch_size,
           n_classes=n_classes, model_type=model_type, month=month, image_suffix='_2019_10.tif',
           codes_to_keep=codes_to_keep, gpu_id=gpu_id)

Setting up a new session...


depth:= 0, nfilters: 16
depth:= 1, nfilters: 32
depth:= 2, nfilters: 64
depth:= 3, nfilters: 128
depth:= 4, nfilters: 256
depth:= 5, nfilters: 512
depth:= 6, nfilters: 256
depth:= 7, nfilters: 128
depth:= 8, nfilters: 64
depth:= 9, nfilters: 32
depth:= 10, nfilters: 16


Validation epoch 1: 100%|██████████| 196/196 [01:11<00:00,  2.75it/s]


Epoch 1:
    Train loss 0.227, accuracy 0.781, F1-score 0.740, MCC: 0.560, Dice: 0.740
    Val loss 0.185, accuracy 0.839, F1-score 0.819, MCC: 0.666, Dice: 0.819


Validation epoch 2: 100%|██████████| 196/196 [01:06<00:00,  2.94it/s]


...

Epoch 98:
    Train loss 0.161, accuracy 0.857, F1-score 0.832, MCC: 0.701, Dice: 0.832
    Val loss 0.161, accuracy 0.856, F1-score 0.845, MCC: 0.708, Dice: 0.845


Training epoch 100:   0%|          | 0/856 [00:00<?, ?it/s]

Epoch 99:
    Train loss 0.162, accuracy 0.855, F1-score 0.830, MCC: 0.699, Dice: 0.830
    Val loss 0.160, accuracy 0.858, F1-score 0.845, MCC: 0.709, Dice: 0.845


Validation epoch 100: 100%|██████████| 196/196 [01:42<00:00,  1.92it/s]

Epoch 100:
    Train loss 0.161, accuracy 0.857, F1-score 0.833, MCC: 0.703, Dice: 0.833
    Val loss 0.160, accuracy 0.858, F1-score 0.842, MCC: 0.707, Dice: 0.842


# Using Han and Burak's original sample locations

In [6]:
def run_france(epochs=100, lr=0.001, lr_decay=None, n_filters=16, batch_size=8,
               n_classes=1, model_type='resunet-d6', month='april', gpu_id=0):
    
    folder_name = model_type+'_'+month+'_nfilter-'+str(n_filters)+'_bs-'+str(batch_size)+'_lr-'+str(lr)
    if lr_decay:
        folder_name = folder_name + '_lrdecay-'+str(lr_decay)
    save_path = '../experiments/france/'+folder_name
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    save_model_name = os.path.join(save_path, "model.params")

    image_directory = '../data/planet/france/'+month
    label_directory = '../data/planet/france/labels/'

    env_name = 'france_'+folder_name
    vis = visdom.Visdom(port=8097, env=env_name)
    
    args = {}
    args['batch_size'] = batch_size
    args['gpu'] = gpu_id
    args['visdom'] = vis
    args['visdom_every'] = 10

    # Load train/val/test splits
    splits_df = pd.read_csv('../data/splits/hanAndBurak_planetImagery_splits.csv')
    train_names = splits_df[splits_df['fold'] == 'train']['image_id'].values
    val_names = splits_df[splits_df['fold'] == 'val']['image_id'].values
    test_names = splits_df[splits_df['fold'] == 'test']['image_id'].values

    train_dataset = PlanetDataset(image_directory, label_directory, image_names=train_names)
    val_dataset = PlanetDataset(image_directory, label_directory, image_names=val_names)
    test_dataset = PlanetDataset(image_directory, label_directory, image_names=test_names)

    train_dataloader = gluon.data.DataLoader(train_dataset, batch_size=batch_size)
    val_dataloader = gluon.data.DataLoader(val_dataset, batch_size=batch_size)
    test_dataloader = gluon.data.DataLoader(test_dataset, batch_size=batch_size)

    # define model
    if model_type == 'resunet-d6':
        model = ResUNet_d6(_nfilters_init=n_filters, _NClasses=n_classes)
    elif model_type == 'resunet-d7':
        model = ResUNet_d7(_nfilters_init=n_filters, _NClasses=n_classes)
    model.initialize()
    model.hybridize()
    model.collect_params().reset_ctx(mx.gpu(args['gpu']))

    # define loss function
    tanimoto_dual = Tanimoto_wth_dual()
    if lr_decay:
        schedule = mx.lr_scheduler.FactorScheduler(step=1, factor=lr_decay)
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr, lr_scheduler=schedule)
    else:
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr)
    trainer = gluon.Trainer(model.collect_params(), optimizer=adam_optimizer)

    # containers for metrics to log
    train_metrics = {'train_loss': [], 'train_acc': [], 'train_f1': [], 
                     'train_mcc': [], 'train_dice': []}
    val_metrics = {'val_loss': [], 'val_acc': [], 'val_f1': [], 
                   'val_mcc': [], 'val_dice': []}
    best_mcc = 0.0

    # training loop
    for epoch in range(1, epochs+1):

        # training set
        train_loss, train_accuracy, train_f1, train_mcc, train_dice = train_model(
            train_dataloader, model, tanimoto_dual, trainer, epoch, args)

        # training set metrics
        train_loss_avg = train_loss / len(train_dataset)
        train_metrics['train_loss'].append(train_loss_avg)
        train_metrics['train_acc'].append(train_accuracy.get()[1])
        train_metrics['train_f1'].append(train_f1.get()[1])
        train_metrics['train_mcc'].append(train_mcc.get()[1])
        train_metrics['train_dice'].append(train_dice.get()[1])

        # validation set
        val_loss, val_accuracy, val_f1, val_mcc, val_dice = evaluate_model(
            val_dataloader, model, tanimoto_dual, epoch, args)

        # validation set metrics
        val_loss_avg = val_loss / len(val_dataset)
        val_metrics['val_loss'].append(val_loss_avg)
        val_metrics['val_acc'].append(val_accuracy.get()[1])
        val_metrics['val_f1'].append(val_f1.get()[1])
        val_metrics['val_mcc'].append(val_mcc.get()[1])
        val_metrics['val_dice'].append(val_dice.get()[1])

        print("Epoch {}:".format(epoch))
        print("    Train loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            train_loss_avg, train_accuracy.get()[1], train_f1.get()[1], train_mcc.get()[1], train_dice.get()[1]))
        print("    Val loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            val_loss_avg, val_accuracy.get()[1], val_f1.get()[1], val_mcc.get()[1], val_dice.get()[1]))

        # save model based on best MCC metric
        if val_mcc.get()[1] > best_mcc:
            model.save_parameters(save_model_name)
            best_mcc = val_mcc.get()[1]

        # save metrics
        metrics = pd.concat([pd.DataFrame(train_metrics), pd.DataFrame(val_metrics)], axis=1)
        metrics.to_csv(os.path.join(save_path, 'metrics.csv'), index=False)

        # visdom
        vis.line(Y=np.stack([train_metrics['train_loss'], val_metrics['val_loss']], axis=1), 
                 X=np.arange(1, epoch+1), win="Loss", 
                 opts=dict(legend=['train loss', 'val loss'], markers=False, title="Losses",
                           xlabel="Epoch", ylabel="Loss")
                )
        vis.line(Y=np.stack([train_metrics['train_mcc'], val_metrics['val_mcc']], axis=1), 
                 X=np.arange(1, epoch+1), win="MCC", 
                 opts=dict(legend=['train MCC', 'val MCC'], markers=False, title="MCC",
                           xlabel="Epoch", ylabel="MCC")
                )


In [7]:
# # ============================ #
# # user-specified hyperparameters
# # ============================ #
# epochs = 100
# lr = 0.001
# lr_decay = None
# n_filters = 16
# batch_size = 8
# n_classes = 1
# model_type = 'resunet-d6'
# month = 'april'
# gpu_id = 0
# # ============================ #

# run_france(epochs=epochs, lr=lr, lr_decay=lr_decay, n_filters=n_filters, batch_size=batch_size,
#            n_classes=n_classes, model_type=model_type, month=month, gpu_id=gpu_id)

In [8]:
def run_multitemporal_france(epochs=100, lr=0.001, lr_decay=None, n_filters=16, batch_size=8,
                             n_classes=1, model_type='resunet-d6',
                             image_directories=['../data/planet/france/january_256x256/',
                                                '../data/planet/france/april_256x256/',
                                                '../data/planet/france/october_256x256/'],
                             label_directory=['../data/planet/france/labels_256x256/'],
                             months='janAprOct',
                             gpu_id=0):
    
    folder_name = model_type+'_'+months+'_nfilter-'+str(n_filters)+'_bs-'+str(batch_size)+'_lr-'+str(lr)
    if lr_decay:
        folder_name = folder_name + '_lrdecay-'+str(lr_decay)
    save_path = '../experiments/france/'+folder_name
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    save_model_name = os.path.join(save_path, "model.params")

    env_name = 'france_'+folder_name
    vis = visdom.Visdom(port=8097, env=env_name)
    
    args = {}
    args['batch_size'] = batch_size
    args['gpu'] = gpu_id
    args['visdom'] = vis
    args['visdom_every'] = 10

    # Load train/val/test splits
    splits_df = pd.read_csv('../data/splits/hanAndBurak_planetImagery_splits.csv')
    train_names = splits_df[splits_df['fold'] == 'train']['image_id'].values
    val_names = splits_df[splits_df['fold'] == 'val']['image_id'].values
    test_names = splits_df[splits_df['fold'] == 'test']['image_id'].values

    train_dataset = PlanetMultitemp(image_directories, label_directory, image_names=train_names)
    val_dataset = PlanetMultitemp(image_directories, label_directory, image_names=val_names)
    test_dataset = PlanetMultitemp(image_directories, label_directory, image_names=test_names)

    train_dataloader = gluon.data.DataLoader(train_dataset, batch_size=batch_size)
    val_dataloader = gluon.data.DataLoader(val_dataset, batch_size=batch_size)
    test_dataloader = gluon.data.DataLoader(test_dataset, batch_size=batch_size)

    # define model
    if model_type == 'resunet-d6':
        model = ResUNet_d6(_nfilters_init=n_filters, _NClasses=n_classes,
                           hsv_outchannels=len(image_directories)*3)
    elif model_type == 'resunet-d7':
        model = ResUNet_d7(_nfilters_init=n_filters, _NClasses=n_classes,
                           hsv_outchannels=len(image_directories)*3)
    model.initialize()
    model.hybridize()
    model.collect_params().reset_ctx(mx.gpu(args['gpu']))

    # define loss function
    tanimoto_dual = Tanimoto_wth_dual()
    if lr_decay:
        schedule = mx.lr_scheduler.FactorScheduler(step=1, factor=lr_decay)
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr, lr_scheduler=schedule)
    else:
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr)
    trainer = gluon.Trainer(model.collect_params(), optimizer=adam_optimizer)

    # containers for metrics to log
    train_metrics = {'train_loss': [], 'train_acc': [], 'train_f1': [], 
                     'train_mcc': [], 'train_dice': []}
    val_metrics = {'val_loss': [], 'val_acc': [], 'val_f1': [], 
                   'val_mcc': [], 'val_dice': []}
    best_mcc = 0.0

    # training loop
    for epoch in range(1, epochs+1):

        # training set
        train_loss, train_accuracy, train_f1, train_mcc, train_dice = train_model(
            train_dataloader, model, tanimoto_dual, trainer, epoch, args)

        # training set metrics
        train_loss_avg = train_loss / len(train_dataset)
        train_metrics['train_loss'].append(train_loss_avg)
        train_metrics['train_acc'].append(train_accuracy.get()[1])
        train_metrics['train_f1'].append(train_f1.get()[1])
        train_metrics['train_mcc'].append(train_mcc.get()[1])
        train_metrics['train_dice'].append(train_dice.get()[1])

        # validation set
        val_loss, val_accuracy, val_f1, val_mcc, val_dice = evaluate_model(
            val_dataloader, model, tanimoto_dual, epoch, args)

        # validation set metrics
        val_loss_avg = val_loss / len(val_dataset)
        val_metrics['val_loss'].append(val_loss_avg)
        val_metrics['val_acc'].append(val_accuracy.get()[1])
        val_metrics['val_f1'].append(val_f1.get()[1])
        val_metrics['val_mcc'].append(val_mcc.get()[1])
        val_metrics['val_dice'].append(val_dice.get()[1])

        print("Epoch {}:".format(epoch))
        print("    Train loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            train_loss_avg, train_accuracy.get()[1], train_f1.get()[1], train_mcc.get()[1], train_dice.get()[1]))
        print("    Val loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            val_loss_avg, val_accuracy.get()[1], val_f1.get()[1], val_mcc.get()[1], val_dice.get()[1]))

        # save model based on best MCC metric
        if val_mcc.get()[1] > best_mcc:
            model.save_parameters(save_model_name)
            best_mcc = val_mcc.get()[1]

        # save metrics
        metrics = pd.concat([pd.DataFrame(train_metrics), pd.DataFrame(val_metrics)], axis=1)
        metrics.to_csv(os.path.join(save_path, 'metrics.csv'), index=False)

        # visdom
        vis.line(Y=np.stack([train_metrics['train_loss'], val_metrics['val_loss']], axis=1), 
                 X=np.arange(1, epoch+1), win="Loss", 
                 opts=dict(legend=['train loss', 'val loss'], markers=False, title="Losses",
                           xlabel="Epoch", ylabel="Loss")
                )
        vis.line(Y=np.stack([train_metrics['train_mcc'], val_metrics['val_mcc']], axis=1), 
                 X=np.arange(1, epoch+1), win="MCC", 
                 opts=dict(legend=['train MCC', 'val MCC'], markers=False, title="MCC",
                           xlabel="Epoch", ylabel="MCC")
                )


In [ ]:
# ============================ #
# user-specified hyperparameters
# ============================ #
epochs = 100
lr = 0.001
lr_decay = None
n_filters = 16
batch_size = 8
n_classes = 1
model_type = 'resunet-d6'
img_dirs = ['../data/planet/france/january_256x256/',
            '../data/planet/france/april_256x256/',
            '../data/planet/france/october_256x256/']
label_dir = '../data/planet/france/labels_256x256/'
months = 'janAprOct'
gpu_id = 0
# ============================ #

run_multitemporal_france(epochs=epochs, lr=lr, lr_decay=lr_decay, n_filters=n_filters,
                         batch_size=batch_size, n_classes=n_classes, model_type=model_type,
                         image_directories=img_dirs, label_directory=label_dir,
                         months=months, gpu_id=gpu_id)

Setting up a new session...
Training epoch 1:   0%|          | 0/155 [00:00<?, ?it/s]

depth:= 0, nfilters: 16
depth:= 1, nfilters: 32
depth:= 2, nfilters: 64
depth:= 3, nfilters: 128
depth:= 4, nfilters: 256
depth:= 5, nfilters: 512
depth:= 6, nfilters: 256
depth:= 7, nfilters: 128
depth:= 8, nfilters: 64
depth:= 9, nfilters: 32
depth:= 10, nfilters: 16


Training epoch 2:   0%|          | 0/155 [00:00<?, ?it/s]

Epoch 1:
    Train loss 0.302, accuracy 0.752, F1-score 0.777, MCC: 0.503, Dice: 0.777
    Val loss 0.220, accuracy 0.768, F1-score 0.772, MCC: 0.528, Dice: 0.772


Validation epoch 2: 100%|██████████| 51/51 [00:30<00:00,  1.68it/s]


...

Training epoch 81:   0%|          | 0/155 [00:00<?, ?it/s]

Epoch 80:
    Train loss 0.150, accuracy 0.864, F1-score 0.871, MCC: 0.712, Dice: 0.871
    Val loss 0.168, accuracy 0.830, F1-score 0.836, MCC: 0.653, Dice: 0.836


Training epoch 82:   0%|          | 0/155 [00:00<?, ?it/s]

Epoch 81:
    Train loss 0.151, accuracy 0.864, F1-score 0.871, MCC: 0.711, Dice: 0.871
    Val loss 0.167, accuracy 0.830, F1-score 0.834, MCC: 0.654, Dice: 0.834


Training epoch 82:  48%|████▊     | 74/155 [01:06<01:13,  1.10it/s]